In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
from pyspark.sql import types

In [3]:
schema = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True) 
    ])

In [4]:
df_raw_test = spark \
            .read \
            .options(header=True) \
            .csv('fhv_tripdata_2019-10.csv')

In [5]:
df_raw_test.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropOff_datetime: string (nullable = true)
 |-- PUlocationID: string (nullable = true)
 |-- DOlocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [6]:
# df_raw = spark \
#             .read \
#             .options(header=True) \
#             .schema(schema) \
#             .csv('fhv_tripdata_2019-10.csv')

In [7]:
df_raw.count()

1897493

In [8]:
df_raw.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [9]:
df_raw \
    .repartition(6) \
    .write \
    .parquet('fhv_tripdata_2019-10', mode='overwrite')

In [7]:
df_parquet = spark \
                .read \
                .parquet('fhv_tripdata_2019-10')

In [19]:
df_parquet.show(5)

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|     B00889         |2019-10-01 15:38:29|2019-10-01 15:56:54|         129|          92|   null|       B00889         |
|              B01239|2019-10-02 10:17:37|2019-10-02 10:33:46|         264|         241|   null|                B01239|
|              B01745|2019-10-01 14:07:24|2019-10-01 14:19:02|         264|         215|   null|                B01745|
|              B00256|2019-10-02 13:05:34|2019-10-02 13:54:04|         264|         264|   null|                B00256|
|              B03060|2019-10-01 13:33:31|2019-10-01 13:49:13|         264|         155|   null|                B02875|
+--------------------+------------------

In [8]:
df_parquet.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [9]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col 
from datetime import datetime, timedelta
from pyspark.sql import udf

# SOLUTION 1:

In [78]:
df_parquet.groupby(F.to_date("pickup_datetime").alias("date")).count().filter(col('date')=="2019-10-15").show()

+----------+-----+
|      date|count|
+----------+-----+
|2019-10-15|62610|
+----------+-----+



In [60]:
df_processing = df_parquet \
                .withColumn('length_trips_hours', 
                            F.round((df_parquet.dropoff_datetime.cast('long') - df_parquet.pickup_datetime.cast('long'))/3600,3)
                            )
                # .withColumn('length_trips_hours', convert_interval_time_udf(df_parquet.length_trips))

In [61]:
df_processing.registerTempTable('fhv_data')

In [62]:
spark.sql("""SELECT MAX(length_trips_hours) FROM fhv_data
""").show(20)

+-----------------------+
|max(length_trips_hours)|
+-----------------------+
|               631152.5|
+-----------------------+



In [63]:
df_taxizone = df = spark.read \
    .option("header", "true") \
    .csv('../taxi_zone_lookup.csv')

In [64]:
df_taxizone.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [65]:
df_taxizone.registerTempTable('taxi_zone')

c:\Users\Windows\anaconda3\envs\dev\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [68]:
spark.sql(""" SELECT t.zone, count(1) from fhv_data as f inner join taxi_zone as t on f.PULocationID=t.LocationID
          group by t.zone order by 2 desc
""").show(20)

+--------------------+--------+
|                zone|count(1)|
+--------------------+--------+
|                  NV| 1500773|
|            Flushing|   12183|
|     Jackson Heights|   10952|
|                  NA|   10235|
|         JFK Airport|    9307|
|Saint George/New ...|    7680|
|   LaGuardia Airport|    7603|
|             Astoria|    7370|
|              Corona|    7175|
|        Midtown East|    6789|
|        North Corona|    5964|
|      Midtown Center|    5880|
|            Elmhurst|    5763|
|Times Sq/Theatre ...|    5451|
|  Murray Hill-Queens|    5306|
|         Old Astoria|    4493|
|          Mount Hope|    3973|
|            Steinway|    3955|
|Van Nest/Morris Park|    3709|
|    Sunset Park East|    3476|
+--------------------+--------+
only showing top 20 rows



# SOLUTION 2

In [69]:
df_parquet.registerTempTable("fhv_data_2")

c:\Users\Windows\anaconda3\envs\dev\lib\site-packages\pyspark\sql\dataframe.py:330: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [72]:
spark.sql(""" Select count(*) from fhv_data_2 where cast(pickup_datetime as date) ="2019-10-15"
""").show(5)

+--------+
|count(1)|
+--------+
|   62610|
+--------+



In [79]:
df_parquet.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [82]:
spark.sql(""" Select pickup_datetime,
                     dropoff_datetime,
                     timestampdiff(hour, pickup_datetime, dropoff_datetime) as hours
            from fhv_data_2 
            order by hours desc
""").show(20)

+-------------------+-------------------+------+
|    pickup_datetime|   dropoff_datetime| hours|
+-------------------+-------------------+------+
|2019-10-28 09:00:00|2091-10-28 09:30:00|631152|
|2019-10-11 18:00:00|2091-10-11 18:30:00|631152|
|2019-10-31 23:46:33|2029-11-01 00:13:00| 87672|
|2019-10-01 21:43:42|2027-10-01 21:45:23| 70128|
|2019-10-17 14:00:00|2020-10-18 00:00:00|  8794|
|2019-10-26 21:26:00|2020-10-26 21:36:00|  8784|
|2019-10-30 12:30:04|2019-12-30 13:02:08|  1464|
|2019-10-25 07:04:57|2019-12-08 07:54:33|  1056|
|2019-10-25 07:04:57|2019-12-08 07:21:11|  1056|
|2019-10-01 07:21:12|2019-11-03 08:44:21|   793|
|2019-10-01 13:47:17|2019-11-03 15:20:28|   793|
|2019-10-01 13:41:00|2019-11-03 14:58:51|   793|
|2019-10-01 08:56:38|2019-11-03 09:25:15|   792|
|2019-10-01 16:28:23|2019-11-03 16:47:21|   792|
|2019-10-01 08:18:17|2019-11-03 08:32:13|   792|
|2019-10-01 15:07:47|2019-11-03 15:13:23|   792|
|2019-10-01 14:18:49|2019-11-03 14:51:25|   792|
|2019-10-01 14:40:26

In [81]:
spark.sql(""" Select pickup_datetime,
                     dropoff_datetime,
                     (unix_timestamp(dropoff_datetime) - unix_timestamp(pickup_datetime))/3600 as hours
            from fhv_data_2 
            order by hours desc
""").show(20)

+-------------------+-------------------+------------------+
|    pickup_datetime|   dropoff_datetime|             hours|
+-------------------+-------------------+------------------+
|2019-10-28 09:00:00|2091-10-28 09:30:00|          631152.5|
|2019-10-11 18:00:00|2091-10-11 18:30:00|          631152.5|
|2019-10-31 23:46:33|2029-11-01 00:13:00| 87672.44083333333|
|2019-10-01 21:43:42|2027-10-01 21:45:23| 70128.02805555555|
|2019-10-17 14:00:00|2020-10-18 00:00:00|            8794.0|
|2019-10-26 21:26:00|2020-10-26 21:36:00| 8784.166666666666|
|2019-10-30 12:30:04|2019-12-30 13:02:08|1464.5344444444445|
|2019-10-25 07:04:57|2019-12-08 07:54:33|1056.8266666666666|
|2019-10-25 07:04:57|2019-12-08 07:21:11|1056.2705555555556|
|2019-10-01 13:47:17|2019-11-03 15:20:28| 793.5530555555556|
|2019-10-01 07:21:12|2019-11-03 08:44:21| 793.3858333333334|
|2019-10-01 13:41:00|2019-11-03 14:58:51|          793.2975|
|2019-10-01 18:43:20|2019-11-03 19:43:13| 792.9980555555555|
|2019-10-01 18:43:46|201